# Kickstarter_survivors
### Sifan Liu

In [41]:
# Load packages and read sample data
import numpy as np
import pandas as pd

in_file = 'source/kickstarter_sample.csv'

In [42]:
sample_data = pd.read_csv(in_file)
sample_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4090 entries, 0 to 4089
Data columns (total 37 columns):
backers_count               4090 non-null int64
blurb                       4088 non-null object
category                    4090 non-null object
converted_pledged_amount    4090 non-null int64
country                     4090 non-null object
created_at                  4090 non-null int64
creator                     4090 non-null object
currency                    4090 non-null object
currency_symbol             4090 non-null object
currency_trailing_code      4090 non-null bool
current_currency            4090 non-null object
deadline                    4090 non-null int64
disable_communication       4090 non-null bool
fx_rate                     4090 non-null float64
goal                        4090 non-null float64
id                          4090 non-null int64
is_starrable                4090 non-null bool
launched_at                 4090 non-null int64
name                 

In [43]:
# convert unix time
time_cols = ['created_at', 'deadline', 'state_changed_at','launched_at']
sample_data[time_cols] = sample_data[time_cols].apply(pd.to_datetime,unit='s')
display(sample_data.head())

,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type,location,friends,is_backing,is_starred,permissions
0,80,I will be an artist-in-residence at Elsewhere ...,"{""urls"":{""web"":{""discover"":""http://www.kicksta...",3596,US,2012-01-07 23:55:05,"{""urls"":{""web"":{""user"":""https://www.kickstarte...",USD,$,True,...,2012-04-19 20:16:00,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",3596.0,domestic,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...",NaN,NaN,NaN,NaN
1,47,We are looking to bring a Visiting Sculptor fr...,"{""urls"":{""web"":{""discover"":""http://www.kicksta...",4117,US,2012-03-02 22:29:22,"{""urls"":{""web"":{""user"":""https://www.kickstarte...",USD,$,True,...,2012-04-20 22:06:38,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",4117.0,domestic,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...",NaN,NaN,NaN,NaN
2,80,Surrealistic oil paintings capturing the metam...,"{""urls"":{""web"":{""discover"":""http://www.kicksta...",3125,US,2012-03-24 14:16:07,"{""urls"":{""web"":{""user"":""https://www.kickstarte...",USD,$,True,...,2012-04-17 03:59:03,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",3125.0,domestic,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...",NaN,NaN,NaN,NaN
3,82,1000 Artists is a public art-making installati...,"{""urls"":{""web"":{""discover"":""http://www.kicksta...",4586,US,2012-03-05 05:41:24,"{""urls"":{""web"":{""user"":""https://www.kickstarte...",USD,$,True,...,2012-05-08 01:22:25,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",4586.0,domestic,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...",NaN,NaN,NaN,NaN
4,31,P.M.A.F.T.W. my upcoming solo show June 2012 a...,"{""urls"":{""web"":{""discover"":""http://www.kicksta...",1036,US,2012-03-23 04:23:27,"{""urls"":{""web"":{""user"":""https://www.kickstarte...",USD,$,True,...,2012-04-03 02:57:23,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",1036.0,domestic,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...",NaN,NaN,NaN,NaN


In [44]:
# factorize state
sample_data['state'] = sample_data['state'].astype('category')
sample_data['state'].describe()

count           4090
unique             5
top       successful
freq            2773
Name: state, dtype: object

In [45]:
# parse json format
import json
df = sample_data[['category','creator','location']]

# 'sub category'
parsed = df['category'].apply(json.loads).apply(pd.Series)
sample_data['subcat'] = parsed['slug']

# 'creator_name'
parsed = df['creator'].apply(json.loads).apply(pd.Series)
sample_data['creator_name'] = parsed['name']

# 'short_name' from location
i=0
for row in df['location']:
    try:
        sample_data.at[i,'city']=json.loads(row)['short_name']
    except TypeError:
        pass
    i+=1   

sample_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4090 entries, 0 to 4089
Data columns (total 40 columns):
backers_count               4090 non-null int64
blurb                       4088 non-null object
category                    4090 non-null object
converted_pledged_amount    4090 non-null int64
country                     4090 non-null object
created_at                  4090 non-null datetime64[ns]
creator                     4090 non-null object
currency                    4090 non-null object
currency_symbol             4090 non-null object
currency_trailing_code      4090 non-null bool
current_currency            4090 non-null object
deadline                    4090 non-null datetime64[ns]
disable_communication       4090 non-null bool
fx_rate                     4090 non-null float64
goal                        4090 non-null float64
id                          4090 non-null int64
is_starrable                4090 non-null bool
launched_at                 4090 non-null datetime6

In [50]:
sample_data['life'] = sample_data['deadline']-sample_data['launched_at']
sample_data['duration'] = sample_data['state_changed_at']-sample_data['launched_at']
sample_data['prep'] =  sample_data['launched_at'] - sample_data['created_at']

sample_data['prep']
#sample_data['state_changed_at'][0].year

0       80 days 21:19:15
1       25 days 23:37:16
2        4 days 08:45:12
3       23 days 19:41:01
4        5 days 22:33:56
5        7 days 01:07:25
6        7 days 12:20:45
7       51 days 06:27:03
8       20 days 13:55:53
9        2 days 19:28:54
10      63 days 18:14:49
11       3 days 23:34:18
12      17 days 07:56:09
13      10 days 22:21:21
14      21 days 15:58:29
15      13 days 11:01:18
16      27 days 20:35:09
17       3 days 10:29:47
18      85 days 19:45:43
19       5 days 02:45:38
20       5 days 02:26:23
21      33 days 18:50:03
22     127 days 04:54:35
23       8 days 01:21:59
24       0 days 01:41:48
25      33 days 16:44:09
26       2 days 21:44:01
27       2 days 08:47:53
28       3 days 23:47:37
29       2 days 22:38:24
              ...       
4060     2 days 03:09:30
4061   164 days 16:49:11
4062    27 days 23:47:17
4063    37 days 18:48:42
4064    25 days 23:15:51
4065   135 days 02:55:41
4066     1 days 21:16:44
4067   157 days 00:03:24
4068    16 days 00:11:22


In [54]:
# keep only US observations
df_sample = sample_data[sample_data.country == "US"]
df_sample.info()

# keep selected columns
col = ['country','backers_count','goal','launched_at','deadline','state_changed_at','state',
       'pledged','city','creator_name','subcat','life','duration','prep']
df_sample = df_sample[col]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3617 entries, 0 to 4089
Data columns (total 43 columns):
backers_count               3617 non-null int64
blurb                       3615 non-null object
category                    3617 non-null object
converted_pledged_amount    3617 non-null int64
country                     3617 non-null object
created_at                  3617 non-null datetime64[ns]
creator                     3617 non-null object
currency                    3617 non-null object
currency_symbol             3617 non-null object
currency_trailing_code      3617 non-null bool
current_currency            3617 non-null object
deadline                    3617 non-null datetime64[ns]
disable_communication       3617 non-null bool
fx_rate                     3617 non-null float64
goal                        3617 non-null float64
id                          3617 non-null int64
is_starrable                3617 non-null bool
launched_at                 3617 non-null datetime6

In [61]:
df_sample.groupby('city').describe()

backers_count                                         \
                               count        mean         std    min     25%   
city                                                                          
Aberdeen, SD                     1.0    1.000000         NaN    1.0    1.00   
Accra, Ghana                     1.0    8.000000         NaN    8.0    8.00   
Akron, OH                        1.0    4.000000         NaN    4.0    4.00   
Alameda, CA                      1.0  118.000000         NaN  118.0  118.00   
Albany, NY                       1.0   52.000000         NaN   52.0   52.00   
Albany, OR                       1.0    4.000000         NaN    4.0    4.00   
Albuquerque, NM                 10.0   60.300000   53.696782    0.0   23.25   
Alexandria, MN                   1.0    0.000000         NaN    0.0    0.00   
Alexandria, VA                   2.0   50.500000   34.648232   26.0   38.25   
Alfred, NY                       6.0   39.000000   19.421637   23.0   25.00   
Alger, MI                        1.0    0.000000         NaN    0.0    0.00   
Allen, TX                        1.0   53.000000         NaN   53.0   53.00   
Allendale, SC                    1.0    8.000000         NaN    8.0    8.00   
Allentown, PA                    2.0    1.000000    0.000000    1.0    1.00   
Alliance, OH                     1.0    0.000000         NaN    0.0    0.00   
Allston, MA                      1.0   16.000000         NaN   16.0   16.00   
Amarillo, TX                     2.0   12.500000   17.677670    0.0    6.25   
American Canyon, CA              1.0  159.000000         NaN  159.0  159.00   
Ames, IA                         1.0   22.000000         NaN   22.0   22.00   
Amherst, MA                      3.0   33.333333   38.370996    5.0   11.50   
Amherst, VA                      1.0   52.000000         NaN   52.0   52.00   
Amityville, NY                   1.0    2.000000         NaN    2.0    2.00   
Amsterdam, Netherlands           3.0   49.000000   32.357379   26.0   30.50   
Anacortes, WA                    1.0   54.000000         NaN   54.0   54.00   
Anacostia, DC                    1.0   24.000000         NaN   24.0   24.00   
Anaheim, CA                      1.0  124.000000         NaN  124.0  124.00   
Anchorage, AK                    5.0   46.000000   57.410800    0.0   20.00   
Andover, MA                      1.0   26.000000         NaN   26.0   26.00   
Angwin, CA                       1.0   31.000000         NaN   31.0   31.00   
Ann Arbor, MI                   16.0   27.875000   19.932804    0.0   18.75   
...                              ...         ...         ...    ...     ...   
White Plains, NY                 1.0   28.000000         NaN   28.0   28.00   
Whitehouse, TX                   1.0    1.000000         NaN    1.0    1.00   
Whittier, CA                     1.0   21.000000         NaN   21.0   21.00   
Wichita, KS                      5.0    9.200000    6.379655    0.0    8.00   
Wilkesboro, NC                   1.0   21.000000         NaN   21.0   21.00   
Willamina, OR                    1.0   23.000000         NaN   23.0   23.00   
Williamsburg, VA                 1.0    8.000000         NaN    8.0    8.00   
Williamsport, PA                 1.0   21.000000         NaN   21.0   21.00   
Williamstown, NJ                 1.0    4.000000         NaN    4.0    4.00   
Williston, ND                    1.0    6.000000         NaN    6.0    6.00   
Willow River, MN                 1.0   10.000000         NaN   10.0   10.00   
Wilmer, TX                       1.0    3.000000         NaN    3.0    3.00   
Wilmington, DE                   3.0   20.666667   11.930353   11.0   14.00   
Wilmington, NC                   2.0   41.000000   36.769553   15.0   28.00   
Wilton, CT                       1.0  199.000000         NaN  199.0  199.00   
Windhoek, Namibia                1.0   10.000000         NaN   10.0   10.00   
Windsor Locks, CT                1.0   12.000000         NaN   12.0   12.

In [60]:
pd.pivot_table(df_sample,index=["city"])

,backers_count,converted_pledged_amount,goal,pledged
city,,,,
"Aberdeen, SD",1.000000,3.000000,800.000000,3.000000
"Accra, Ghana",8.000000,350.000000,12000.000000,350.000000
"Akron, OH",4.000000,125.000000,9000.000000,125.000000
"Alameda, CA",118.000000,6761.000000,5500.000000,6761.000000
"Albany, NY",52.000000,543.000000,300.000000,543.000000
"Albany, OR",4.000000,12.000000,111.000000,12.000000
"Albuquerque, NM",60.300000,5715.800000,7000.000000,5715.850000
"Alexandria, MN",0.000000,0.000000,100000.000000,0.000000
"Alexandria, VA",50.500000,2949.000000,2300.000000,2949.000000
